In [11]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

"\nChatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.\n\nImagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.\n\nCreate a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.\n\nThe final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.\n\nExample\nA: Hello MedAssistant.\nB: Hello. How can I help you?\nA: I don't feel well.\nB: Do you have any symptoms?\nA: I have cough and nausea.\nB: Do you want to book an appointment?\nA: Yes, please, for tomorrow.\n\n\nInfo:\n- Feel free to change or arrange a new dataset of intents\n- Try experimenting and tunin

In [4]:
import numpy as np
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet


In [5]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

In [6]:
import numpy as np
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# STEP 0: find intents patterns

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))



In [7]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD


ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(patterns), "patterns")
print(len(tags), "tags")
print(len(all_words), "unique stemmed words")

41 patterns
8 tags
89 unique stemmed words


In [8]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD

num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?
# TBD
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model


In [10]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

Epoch [100/1000], Loss: 0.5667
Epoch [200/1000], Loss: 0.0208
Epoch [300/1000], Loss: 0.0003
Epoch [400/1000], Loss: 0.0001
Epoch [500/1000], Loss: 0.0010
Epoch [600/1000], Loss: 0.0003
Epoch [700/1000], Loss: 0.0009
Epoch [800/1000], Loss: 0.0032
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0011


In [11]:
# STEP 5: Save the model

# TBD: name and save the model
file = "trained.pth"
torch.save(data, file)

print(f'training complete. file saved!')


training complete. file saved!


In [13]:

import random
import numpy as np
import json


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# STEP 6: Test the model
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))



ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))


# TBD: Load the intents file
with open('intents.json', 'r') as f:
    intents = json.load(f)

# TBD: Retrieve the model and all the sizings

# TBD: build the NN
data = torch.load("trained.pth")

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)
botname = 'siri'
print('lets chat')

while True:
    sentence = input('you : ')
    if sentence == "exit":
        print("thanks for joining me!")
        break
    else:
        print('you: ' + sentence)
    
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)

    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
    probs =torch.softmax(output, dim =1)
    prob = probs[0][predicted.item()]
    #print(prob)
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                print(f"{botname}: {random.choice(intent['responses'])}")
    else:
                print(f"{botname}: i do not understand. try to be more specific pls")

lets chat
you: hey
siri: Hello, thanks for visiting
you: whats up
siri: May we offer you a slice of hot pizza?
thanks for joining me!
